# PyKOALA Cubing 

## Table of contents:

1. [Importing class](#importing-class)
2. [Organising data](#organising-data)
3. [KOALA cubing](#KOALA-cubing)
    - [`Cubing` methods](#cubing-methods)
        - [Rows and columns](#rows-and-columns)
        - [`rss_intensity`](#rss_intensity)
        - [`rss_variance`](#rss_variance)
        - [`get_centre_of_mass`](#get_centre_of_mass)
        - [`get_integrated_light_frac`](#get_integrated_light_frac)
4. [Plotting cube data](#plotting-cube-data)

## Importing class

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
from astropy import units as u
from pykoala.corrections.astrometry import AstrometryCorrection

# Pykoala modules

from pykoala.cubing import build_wcs, CubeInterpolator, CubeStacking
from pykoala.instruments.koala_ifu import koala_rss
from pykoala.corrections.astrometry import find_centroid_in_dc
from pykoala.corrections.astrometry import AstrometryCorrection

## Organising data

In [ ]:
# List of RSS objects

std_star_rss = []
data_path = '../data/koala/'
grating = '385R'

for i in [28, 29, 30]:
    filename = os.path.join(data_path, grating, f"27feb200{i}red.fits")
    rss = koala_rss(filename)
    std_star_rss.append(rss)

star_name = rss.info['name'].split(' ')[0]
print("Star name: ", star_name)

In [ ]:

astrom_corr = AstrometryCorrection()

offsets, fig = astrom_corr.register_centroids(std_star_rss, object_name=star_name,
                                         qc_plot=True, centroider='gauss')
for offset in offsets:
    print("Offset (ra, dec) in arcsec: ", offset[0].to('arcsec'), offset[1].to('arcsec'))

## `KOALA` cubing

In [ ]:
datacube_shape = (std_star_rss[0].wavelength.size, 40, 60)
ref_position = (std_star_rss[0].wavelength[0], np.mean(std_star_rss[0].info['fib_ra']), np.mean(std_star_rss[0].info['fib_dec']))  # (deg, deg)
spatial_pixel_size = 1.0 << u.arcsec
spectral_pixel_size = std_star_rss[0].wavelength[1] - std_star_rss[0].wavelength[0]  # (angstrom)

print(f"Creating a WCS with\n position: {ref_position}\n Spatial pixel size: {spatial_pixel_size}\n Spectral pixel size: {spectral_pixel_size}")

wcs = build_wcs(datacube_shape=datacube_shape,
                reference_position=ref_position,
                spatial_pix_size=spatial_pixel_size,
                spectra_pix_size=spectral_pixel_size,
            )

In [ ]:
interpolator = CubeInterpolator(rss_set=std_star_rss)
cube = interpolator.build_cube()
white_image = np.nanmean(cube.intensity, axis=0)

### `Cubing` methods

#### Rows and columns

In [ ]:
print(f"Number of spaxel columns: {cube.n_cols}")
print(f"Number of spaxel rows: {cube.n_rows}")

#### `rss_intensity`

In [ ]:
print(f"Intensity data: \n\n {cube.rss_intensity}")

#### `rss_variance`

In [ ]:
print(f"Intensity data: \n\n {cube.rss_variance}")

#### `get_centre_of_mass`

In [ ]:
print(f"Center of mass of the data cube: \n\n {cube.get_centre_of_mass()}")

## Plotting cube data

In [ ]:
datacube_shape = (std_star_rss[0].wavelength.size, 40, 60)
ref_position = (std_star_rss[0].wavelength[0], np.mean(std_star_rss[0].info['fib_ra']), np.mean(std_star_rss[0].info['fib_dec']))  # (deg, deg)
spatial_pixel_size = 1.0 << u.arcsec
spectral_pixel_size = std_star_rss[0].wavelength[1] - std_star_rss[0].wavelength[0]  # (angstrom)

print(f"Creating a WCS with\n position: {ref_position}\n Spatial pixel size: {spatial_pixel_size}\n Spectral pixel size: {spectral_pixel_size}")

wcs = build_wcs(datacube_shape=datacube_shape,
                reference_position=ref_position,
                spatial_pix_size=spatial_pixel_size,
                spectra_pix_size=spectral_pixel_size,
            )

In [ ]:

pos_com = find_centroid_in_dc(cube, centroider='com', com_power=1.)
pos_com_3 = find_centroid_in_dc(cube, centroider='com', com_power=3.)
pos_gauss = find_centroid_in_dc(cube, centroider='gauss')

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(111, projection=wcs.celestial)
mappable = ax.imshow(np.log10(white_image.value), vmin=-2)
fig.set_size_inches(18.5, 10.5)
plt.colorbar(mappable)
ax.scatter(pos_com.ra, pos_com.dec, marker='*', ec='r', transform=ax.get_transform('world'))
ax.scatter(pos_com_3.ra, pos_com_3.dec, marker='*', ec='lime', transform=ax.get_transform('world'))
ax.scatter(pos_gauss.ra, pos_gauss.dec, marker='+', ec='k', transform=ax.get_transform('world'))
